# Application of amplitude estimation to Finances: Call Option under Black Scholes model

Present Notebook will use **amplification amplitude** techniques as explained in notebook: *08_ApplicationTo_Finance_01_StandardApproach* for solving the price of a *vanilla european Call Option* under the **Black Scholes** model. 

We have developed the *classical_finance* module inside the package *utils* of the *QQuantLib*  (**QQuantLib/utils/classical_finance.py**) where several *Finance* functions were created.



In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

## 1. The problem

We have a **financial asset** whose value at a time $t$ is $S(t)$. The volatility of the asset will be $\sigma$. The **risk free rate** will be $r$. Under this considerations we are going to create a **derivative contract** based on the evolution of the underlying wich a definite duration time (**maturity**): $T$. This product will be characterized by its **return** that will be a function that will depends of the price of the underlying and of the *maturity*:  $f=f(S(t), T)$. The question is: **What is the 'fair' price of our derivative contract at a time t when underlying is S(t): $V_f(S(t), t)$?**.

So we have:

* $S = S(t)$ price of the underlying at $t$.
* $\sigma$ volatility of the underlying.
* *Risk free rate*: $r$
* Return of the financial product: $f=f(S(t), T)$.

And we want:

* $V_f(S(t), t)$


## 2. Black-Scholes Model

One model for obtaining this $V_f(S(t), t)$ is the **Black-Scholes** which relies in the following assumptions:

1. The underlying $S(t)$ follows a lognormal random walk.
2. The risk-free interest rate $r$ is a known function of time $t$.
3. There are no divindends on the underlying $S$
4. Dynamic **delta Hedging** (risk elimination strategy)
5. No arbitrage oppotunities.

Under this condition the Black-Scholes equation relates underlying $S$ with the price of the financial product $V$:

$$\frac{\partial V}{\partial t}+\frac{1}{2}\sigma^2S^2\frac{\partial^2 V}{\partial^2 S} + rS\frac{\partial V}{ \partial S} - rV = 0$$

## 3. Price of derivative contract

So far we have:

1. Financial asset with value $S(t)$, volatility $\sigma$, and risk free rate $r$
2. **Derivative contract**  over the financial asset $S$ with a maturity $T$ and a return of $f=f(S(t), T)$

Under the **Black-Scholes** equation, the value of our **derivative contract**  at time $t$ and correspondient underlying $S(t)$ will be given by the following equation:

$$V(t, S(t)) = e^{r(T-t)} \mathbb{E}_{P_{BS}}[f(S_T)]$$

Where 

* $S_T$ is the value of the underlying asset at the *maturity* time $T$.
* $\mathbb{E}_{P_{BS}}[f(S_T)]$ is the expected value of the return of the *financial product* at maturity time $T$ and asset value $S_T$ when $S_T$ follows a **Black-Scholes** probability density, $P_{BS}$.

The **Black-Scholes** probability density, $P_{BS}$ is given by:

$$P_{BS} (S_T; t, S(t)) = \frac{1}{S_T\sigma\sqrt{2\pi(T-t)}}exp\big({-\frac{(\log S_T -\mu)^2}{2\sigma^2(T-t)}}\big)$$

and

$$\mu = (r-\frac{1}{2}\sigma^2)(T-t)+\log S(t)$$

The $P_{BS}$ give us the probability density of the underlying value at the maturity time $T$, $S_T$, when for a time $t$ the underlaying have a value $S(t)$ (following the **Black-Scholes** equation).

So the price of the financial product for $t$ and $S(t)$  will be:

$$V(t, S(t))= e^{r(T-t)} \mathbb{E}_{P_{BS}}[f] =  e^{r(T-t)} \int_0^\infty P_{BS} (S_T; t, S(t)) f(S_T)dS_T$$


The *return* of the financial product at the maturity time is the **Payoff** of the product: $f(S_T) = Payoff(S_T)$.


First we are going to load the **Black-Scholes** probability density. For this we can use the *bs_probability* function from the  *classical_finance* module (**QQuantLib/utils/classical_finance.py**)

In [ ]:
from QQuantLib.utils.classical_finance import bs_probability

In [ ]:
#Parameters of out model
S0 = 2
r = 0.04
sigma = 0.1
#maturity time for the derivative contract
T = 300/365 #300 days

In [ ]:
n_qbits = 5
x = np.linspace(1,3,2**n_qbits)
prob_bs = bs_probability(s_t=x, s_0=S0,risk_free_rate=r,volatility=sigma,maturity=T)

In [ ]:
#Black-Scholes Probability density
plt.plot(x, prob_bs)
plt.xlabel(r'$S_T$')
plt.ylabel(r'$P(S_T, S_0)$')
plt.title(r'Black-Scholes Probability density. r= {} $\sigma$= {}. $S_0$= {}'.format(r, sigma, S0))

So in the before graph we plot the probability of the value of the underlying S at maturity time T, following the **Black Scholes** model when risk free rate is $r$ and volatility of S is $\sigma$ and the initial value of the underlying is $S_0$

## 4. European Vanilla Call Option

We are going to use as **derivative contract** a Vanilla European Call Option. This derivative allows the holder to buy an underlying asset S at a maturity time $T$ at a fixed price $K$ (**strike**) independently of the value of $S_T$. The return of this option at a time $t$ is:

$$f(t, K) = \max(0, S(t)-K)$$

And the final payoff (the return at the maturity time $T$) will be:

$$Payoff(S_T, T, K)= \max(0, S_T-K)=(S_T-K)^+$$

We are going to use the function *call_payoff* from **QQuantLib/utils/classical_finance** module for the final payoff of the Europan call option.

In [ ]:
from QQuantLib.utils.classical_finance import call_payoff

In [ ]:
#Strike
K = 1.9
payoff_call = call_payoff(x, strike=K)

In [ ]:
#European Call Option Payoff
plt.plot(x, payoff_call)
plt.xlabel(r'$S_T$')
plt.ylabel(r'$Payoff(S_T, S_0)$')
plt.title(r'European Call Option Payoff. K= {}'.format(K))

## 5. Call Option under Black-Scholes

Under the *Black-Scholes* probability density, an asset volatility $\sigma$ and free risk-rate $r$ the price of a call option at $t=0$ and $S(t=0)=S_0$ will be:

$$V(S_0, K, T)=  e^{rT} \int_0^\infty P_{BS} (S_T; S_0) Payoff(S_T, K, T)dS_T=e^{rT} \int_0^\infty P_{BS} (S_T; S_0)(S_T-K)^+dS_T$$

where

$$P_{BS} (S_T; S_0) = \frac{1}{S_T\sigma\sqrt{2\pi T}}exp\big({-\frac{(\log S_T -\mu)^2}{2\sigma^2 T}}\big)$$

and 
$$\mu = (r-\frac{1}{2}\sigma^2)T+\log S_0$$

The value for an European Call Option under the **Black-Scholes** model has an exact solution:
 
$$V(S(t), t)= S(t)*\Phi(d_1)-Ke^{-r*(T-t)}\Phi(d_2)$$
 
where:

$$d_1 = \frac{log(\frac{S(t)}{K})+(r+\frac{1}{2}\sigma^2)(T-t)}{\sigma\sqrt{T-t}}$$
$$d_2 = \frac{log(\frac{S(t)}{K})+(r-\frac{1}{2}\sigma^2)(T-t)}{\sigma\sqrt{T-t}}$$
$$\Phi(x)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{x} e^{\frac{-1}{2}\phi^2}d\phi$$


The function *bs_call_price* from **QQuantLib/utils/classical_finance** module computes this exact solution.

In [ ]:
from QQuantLib.utils.classical_finance import bs_call_price

In [ ]:
exact_price = bs_call_price(s_0=S0, risk_free_rate=r, volatility=sigma, maturity=T, strike=K)
print("Exact price: ",exact_price)

## 6. Quantum Amplification techniques

So for the *Call Option* under the **Black Scholes** model the price will be:

$$V(S_0, K, T)= e^{rT} \int_0^\infty P_{BS} (S_T; S_0)(S_T-K)^+dS_T$$

The integral is the expectation value of the payoff under the **Black-Scholes** probaility density so we can reformulate before equation as:

$$V(S_0, K, T)=  e^{-rT}\mathbb{E}_{P_{BS}}[(S_T-K)^+]$$

We can compute the $\mathbb{E}_{P_{BS}}[(S_T-K)^+]$ in a classical way 

$$\mathbb{E}_{P_{BS}}[(S_T-K)^+] = \sum_{i=0}^{2^n-1} P_{BS}(x_i)(x_i-K)^+$$



In [ ]:
classical_price = np.sum(prob_bs*payoff_call)*np.exp(-r*T)
print("Classical Price: ",classical_price)

$\mathbb{E}_{P_{BS}}[(S_T-K)^+]$ can be calculated using **amplitude amplification** techniques as shown in *08_ApplicationTo_Finance_01_StandardApproach*. Following parts explain how to do this.

### 6.1 Normalisation

First thing we need to do is check if our probability ($P_{BS}(S_t)$) and our pay off function ($Payoff(S_t, T, K)$) are properly normalised:

* For $P_{BS}(x)$ is mandatory that: $\sum_{i=0}^{2^{n}} P_{BS}(x_i) = 1$
* For $Payoff(x, T, K)$ is mandatory that: $Payoff(x_i) \leq 1 \forall i$

If one of the before conditions is not fullfilled we need to normalised the functions!!


In [ ]:
print('p(x) condition: {}'.format(np.sum(prob_bs) == 1))
print('f(x) condition: {}'.format(np.max(payoff_call) <= 1))
#Normalisation of Payy Off
payoff_normalisation = np.max(payoff_call)

payoff_call_normalised = payoff_call/payoff_normalisation
print('f(x) condition: {}'.format(np.max(payoff_call_normalised) <= 1))

We need to used *payoff_call_normalised* instead of *payoff_call* as function for our **amplitude amplification** problem

### 6.2 Loading Data and Oracle creation

Second step will be the loading of the functions into the quantum state:

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array

In [ ]:
p_gate = load_probability(prob_bs)
f_gate = load_array(np.sqrt(payoff_call_normalised))


call_option_oracle = qlm.QRoutine()
call_option_registers = call_option_oracle.new_wires(f_gate.arity)
call_option_oracle.apply(p_gate, call_option_registers[:p_gate.arity])
call_option_oracle.apply(f_gate, call_option_registers)

%qatdisplay call_option_oracle --depth 0 --svg

### 6.3 Amplitude Amplification routines

Now we can use the *amplitude amplification* routines for solving the **amplitude amplification** problem. Following routines will be used:

1. Maximum Likelihood Amplitude Estimation (MLAE)
2. Amplitude Estimation using classical Quantum Phase Estimation (PE_QFT_AE)
3. Amplitude Estimation using iterative Quantum Phase Estimation (IQPE_AE)
4. Iterative Quantum Amplitude Estimation (IQAE)


For using the different classes for solving our call option price problem we need to provide always:

* Oracle (*call_option_oracle*)
* target: $|0\rangle$ state of the last qbit
* index: index of the last qbit

In [ ]:
target = [0]
index = [call_option_oracle.arity-1]

#### MLAE

In [ ]:
from QQuantLib.AE.maximum_likelihood_ae import MLAE

mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True  
}
mlae = MLAE(
    call_option_oracle,
    target = target,
    index = index, 
    **mlae_dict
)

mlae_a = mlae.run()
print('mlae_a: ', mlae_a)

In [ ]:
mlae.circuit_statistics

###  CQPE_AE

In [ ]:
from QQuantLib.AE.ae_classical_qpe import CQPEAE

ae_cqpe_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 8,
    'shots': 100,
    'mcz_qlm': True      
}

ae_cqpe = CQPEAE(
    call_option_oracle,
    target = target,
    index = index, 
    **ae_cqpe_dict
)
ae_cqpe_a  = ae_cqpe.run()

print('ae_cqpe_a: ', ae_cqpe_a)

In [ ]:
ae_cqpe.circuit_statistics

#### IQPE_AE

In [ ]:
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 8,
    'shots': 10,
    'mcz_qlm': True  
}

ae_iqpe = IQPEAE(
    call_option_oracle,
    target = target,
    index = index, 
    **ae_iqpe_dict
)

ae_iqpe_a  = ae_iqpe.run()

print('ae_iqpe_a: ', ae_iqpe_a)

In [ ]:
ae_iqpe.run_time

In [ ]:
ae_iqpe.circuit_statistics

#### IQAE

In [ ]:
from QQuantLib.AE.iterative_quantum_ae import IQAE

iqae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True      
}

iqae = IQAE(
    call_option_oracle,
    target = target,
    index = index, 
    **iqae_dict
)

iqae_a = iqae.run()

print('iqae_a: ', iqae_a)

In [ ]:
iqae.circuit_statistics

### 6.4 Getting the results

The *run* method of the different *amplitude amplification* algorithms used allways return the $a$ estimation (see notebook 08_ApplicationTo_Finance_01_StandardApproach).

For the european call option under **Black Scholes** we normalized the payoff (section 6.1), so, for getting the correct results wee need to undo this normalization:

$$\mathbb{E}_{P_{BS}}[(S_T-K)^+] = a*\max(Payoff(x_i))$$  

Additionally for calculating the the price of the call option under **Black Scholes** we need:

$$V(S_0, K, T)=  e^{-rT}\mathbb{E}_{P_{BS}}[(S_T-K)^+]$$

## SUMMARY

In [ ]:
methods = ['MLAE', 'CQPEAE', 'IQPEAE', 'IQAE']
a_estimated = [mlae.ae, ae_cqpe.ae, ae_iqpe.ae, iqae.ae]


dic_staff = {
    'AE_a': a_estimated,
}

Results = pd.DataFrame(dic_staff, index=methods)

#For getting the proper results we need to undo the payoff_normalisation

Results['AE_Results'] = Results['AE_a']*payoff_normalisation
Results['AE_call_option_price'] = Results['AE_Results']*np.exp(-r*T)
Results['Exact_call_option_price'] = exact_price
Results['classical_price'] = classical_price
Results['Error'] = np.abs(Results['Exact_call_option_price']-Results['AE_call_option_price'])

In [ ]:
Results